In [20]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import numpy as np
from sklearn import preprocessing, cluster, decomposition
from scipy.cluster.vq import kmeans,vq
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.decomposition import FastICA, PCA
from sklearn import metrics

riotapi.set_region("OCE")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(True)

client = MongoClient()
loladb = client.loladb_oce
summoners_collection = loladb.top_summoners
matches_collection = loladb.top_matches
print summoners_collection.count()
print matches_collection.count()

TRAIN_NUM = 4000
TEST_NUM = 1500

126
5751


In [2]:
top_summoners = [s['playerOrTeamId'] for s in summoners_collection.find()]
print len(top_summoners)

126


In [21]:
from sets import Set

summoner_set = Set()

for a_idx, a_match in enumerate(matches_collection.find()):
    if a_idx % 1000  == 0:
        print a_idx
        
    if a_idx == TRAIN_NUM:
        break
        
    participants = a_match['participants'] 
    for a_participant_idx,a_participantIdentity in enumerate(a_match['participantIdentities']):
        summoner_Id = a_participantIdentity['player']['summonerId']
        summoner_set.add(summoner_Id)
        
match_size = matches_collection.count()
summoner_size = len(summoner_set)
print summoner_size

0
1000
2000
3000
4000
3293


In [22]:

X = np.zeros((match_size, summoner_size))
Y = np.zeros(match_size)

summoner_id2idx = dict()
summoner_idx2id = dict()

for a_idx, a_summoner in enumerate(summoner_set):
    summoner_id = str(a_summoner)
    summoner_id2idx[summoner_id] = a_idx
    summoner_idx2id[a_idx] = summoner_id


def Summoner_GetIdxFromId(a_id):
    return summoner_id2idx[a_id]

def Summoner_GetIdFromIdx(a_idx):
    return summoner_idx2id[a_idx]

In [28]:
#Fill the kill difference based on match data
#the summoner in blue team has a '+1', the summoner in red team has a '-1'
real_size = 0
for a_idx, a_match in enumerate(matches_collection.find()):
    if a_idx % 1000  == 0:
        print a_idx
        
    if a_match['matchVersion'].startswith('6.24') == False:
        continue
    
    winning_side = -1
    if a_match['teams'][0]['winner'] == True and a_match['teams'][1]['winner'] == False:
        winning_side = 0
    if a_match['teams'][0]['winner'] == False and a_match['teams'][1]['winner'] == True:
        winning_side = 1 
    if winning_side == -1:
        continue #do not count draw game, if any

    a_duration = a_match['matchDuration']
    a_minute = a_duration/60
    if a_minute < 20:
        continue #do not count remake game
    
    participants = a_match['participants']  
    for a_participant_idx,a_participantIdentity in enumerate(a_match['participantIdentities']):
        summoner_Id = a_participantIdentity['player']['summonerId']

        if str(summoner_Id) in summoner_id2idx.keys():
            participantIdx = summoner_id2idx[str(summoner_Id)]

            my_side = 0
            if a_participant_idx >= 5:
                my_side = 1
            isWinning = my_side == winning_side
            
            Y[real_size] = (winning_side - 0.5) * -2 #change {0, 1} to {1, -1}
            
            if my_side == 0:
                X[real_size][participantIdx] = 1
            else:
                X[real_size][participantIdx] = -1      
    real_size += 1

0
1000
2000
3000
4000
5000


In [26]:
from sklearn import datasets, linear_model

regr = linear_model.LogisticRegression()

TrainX = X[0:4000, :]
TrainY = Y[0:4000]
TestX = X[4000:5500, :]
TestY = Y[4000:5500]

regr.fit(TrainX, TrainY)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
print('Coefficients: \n', regr.coef_[0:50])

Y_eval = regr.predict(TrainX) > 0
Y_correct = Y_eval == (TrainY > 0)
print Y_eval[0:10]
print Y[0:20]
print Y_correct[0:20]
print sum(Y_correct)/float(len(Y_correct))

('Coefficients: \n', array([[ 0.47683817, -0.40073978, -0.41403649, ...,  1.00830357,
        -0.48917207,  0.09644641]]))
[False  True  True  True  True False False False False  True]
[-1.  1.  1.  1.  1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1.  1. -1. -1.
 -1.  1.]
[ True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True]
0.849


In [60]:
weakest_blue = np.min((TestX[:, :] == 1) * regr.coef_, axis = 1)
weakest_red = np.min((TestX[:, :] == -1) * regr.coef_, axis = 1)
PreY = weakest_blue > weakest_red
Y_correct = PreY == (TestY > 0)
print sum(Y_correct)/float(len(Y_correct))

0.524


In [45]:
#use the training results to predict testing matches
TestY_Predict = regr.predict(TestX)
TestY_Correct = TestY_Predict == TestY
print sum(TestY_Correct)
print float(len(TestY))
X_nonempty = TestX != 0
X_sum = np.sum(X_nonempty, axis = 1)
X_hist = np.zeros(11)
X_total = np.zeros(11)
for a_idx, x in enumerate(TestX):
    a_sum = X_sum[a_idx]
    X_hist[a_sum] += TestY_Correct[a_idx]
    X_total[a_sum] += 1

806
1500.0


In [48]:
print X_hist
print X_total

[   0.    0.    3.    2.    5.    7.   15.   23.   57.  157.  537.]
[   0.    1.    4.    5.    9.   15.   21.   56.   99.  305.  985.]


In [85]:
print regr.coef_.shape
for i in range(3574):
    if X[49, i] != 0:
        print summoner_idx2id[i]
        print X[49, i] * regr.coef_[i]
print Y[49]


(3574,)
221515
1.13518934068e+12
221282
-1.13518934068e+12
560308
-1.13518934068e+12
601923
1.13518934068e+12
471029
-1.13518934068e+12
1332520
-1.13518934068e+12
8500155
1.13518934068e+12
276531
-1.13518934068e+12
1890499
1.13518934068e+12
6970167
1.13518934068e+12
-1.0


In [46]:
for a_idx, a_match in enumerate(matches_collection.find()):
    if a_idx ==49:
        print MyCommon.JSONEncoder().encode(a_match)

{"participantIdentities": [{"player": {"profileIcon": 7, "summonerId": 560308, "matchHistoryUri": "/v1/stats/player_history/NA/32503465", "summonerName": "FBI WORST ADC"}, "participantId": 1}, {"player": {"profileIcon": 27, "summonerId": 221282, "matchHistoryUri": "/v1/stats/player_history/NA1/207859013", "summonerName": "swathe two"}, "participantId": 2}, {"player": {"profileIcon": 1386, "summonerId": 471029, "matchHistoryUri": "/v1/stats/player_history/NA1/202120134", "summonerName": "x Luke"}, "participantId": 3}, {"player": {"profileIcon": 544, "summonerId": 276531, "matchHistoryUri": "/v1/stats/player_history/NA1/201020435", "summonerName": "Le Monsieur"}, "participantId": 4}, {"player": {"profileIcon": 1149, "summonerId": 1332520, "matchHistoryUri": "/v1/stats/player_history/NA1/212179429", "summonerName": "Kizutsukanai"}, "participantId": 5}, {"player": {"profileIcon": 685, "summonerId": 1890499, "matchHistoryUri": "/v1/stats/player_history/OC1/200829333", "summonerName": "Zelda

In [92]:
#draw a line using its factor and tier
for a_idx,a_summoner in enumerate(summoner_set):
    if str(a_summoner) in top_summoners:
        participantIdx = summoner_id2idx[str(a_summoner)]
        print a_summoner
        print participantIdx
        print regr.coef_[participantIdx]

229379
0
-1.13518934068e+12
6840379
21
-1.13518934068e+12
6062154
29
-1.13518934068e+12
270436
41
-1.13518934068e+12
221417
103
-1.13518934068e+12
483618
123
-1.13518934068e+12
237869
128
-1.13518934068e+12
4791006
133
-1.13518934068e+12
5202241
138
-1.13518934068e+12
1530604
172
-1.13518934068e+12
483862
237
-1.13518934068e+12
221282
256
-1.13518934068e+12
631376
259
-1.13518934068e+12
303763
298
-1.13518934068e+12
3171338
456
-1.13518934068e+12
3220590
502
-1.13518934068e+12
323773
503
-1.13518934068e+12
222477
576
-1.13518934068e+12
771624
701
-1.13518934068e+12
3262016
714
-1.13518934068e+12
493127
720
-1.13518934068e+12
747148
746
-1.13518934068e+12
820884
750
-1.13518934068e+12
411324
766
-1.13518934068e+12
583483
818
-1.13518934068e+12
1321744
820
-1.13518934068e+12
571021
827
-1.13518934068e+12
231252
828
-1.13518934068e+12
3663783
863
-1.13518934068e+12
6170605
893
-1.13518934068e+12
632903
928
-1.13518934068e+12
313480
955
-1.13518934068e+12
952670
1049
-1.13518934068e+12
236

In [93]:
print regr.coef_[0:50]

[ -1.13518934e+12  -1.13518934e+12  -1.13518934e+12  -1.13518934e+12
   3.10445465e+13  -1.13518934e+12  -1.13518934e+12  -1.13518934e+12
   1.31513366e+13  -1.13518934e+12  -1.13518934e+12  -1.13518934e+12
  -1.13518934e+12  -1.13518934e+12   3.78381249e+13  -1.13518934e+12
  -1.13518934e+12   7.23716727e+13  -4.43443584e+13  -1.13518934e+12
  -1.13518934e+12  -1.13518934e+12  -1.13518934e+12  -7.13967551e+12
  -1.13518934e+12  -1.13518934e+12  -1.13518934e+12  -1.13518934e+12
  -2.35297303e+13  -1.13518934e+12  -1.13518934e+12  -1.13518934e+12
  -1.13518934e+12  -1.13518934e+12  -1.13518934e+12  -1.13518934e+12
  -1.13518934e+12  -1.13518934e+12   6.34081394e+13  -1.13518934e+12
  -1.13518934e+12  -1.13518934e+12  -1.13518934e+12   1.35701702e+13
  -1.13518934e+12  -3.71368296e+13  -1.13518934e+12  -1.13518934e+12
   3.97401088e+13  -1.13518934e+12]
